In [39]:
# Import Dependencies
import pandas as pd
import re
import numpy as np
#from clean import clean

## Import Dataset

In [40]:
twitter_df = pd.read_csv("Resources/sent_analysis_dataset.csv", error_bad_lines=False)

b'Skipping line 8836: expected 4 fields, saw 5\n'
b'Skipping line 535882: expected 4 fields, saw 7\n'


In [41]:
# For right now, we will only use part of the set
twitter_df = twitter_df[:-1500000]
len(twitter_df.index)

78612

In [42]:
twitter_df.head()

,ItemID,Sentiment,SentimentSource,SentimentText
0,1,0,Sentiment140,is so sad for my APL frie...
1,2,0,Sentiment140,I missed the New Moon trail...
2,3,1,Sentiment140,omg its already 7:30 :O
3,4,0,Sentiment140,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,Sentiment140,i think mi bf is cheating on me!!! ...


## Clean Tweets

In [43]:
# Drop and rename columns
twitter_df = twitter_df.drop(["ItemID", "SentimentSource"], axis = 1)
twitter_df = twitter_df.rename(columns = {"SentimentText": "Text"})

In [44]:
# Check columns for missing data
twitter_df.isnull().sum()

Sentiment    0
Text         0
dtype: int64

In [45]:
# Verify data are of correct type
twitter_df.dtypes

Sentiment     int64
Text         object
dtype: object

In [46]:
# Verify sentiment column has appropriate data
twitter_df["Sentiment"].unique()

array([0, 1])

In [47]:
# Clean data using clean function

# twitter_df["Sentiment"] = twitter_df["Sentiment"].map(lambda x: clean(x))

In [48]:
# GET RID OF THIS ONCE WE HAVE CLEAN FUNCTION
twitter_df["Text"] = twitter_df["Text"].map(lambda x: re.sub(r"[!#$%&'\\()*+,-./:;<=>?@\^_`{|}~]", "", x))
twitter_df["Text"] = twitter_df["Text"].map(lambda x: re.sub("\[", " ", x))
twitter_df["Text"] = twitter_df["Text"].map(lambda x: re.sub("\]", " ", x))

In [49]:
twitter_df.head(20)

,Sentiment,Text
0,0,is so sad for my APL friend
1,0,I missed the New Moon trailer
2,1,omg its already 730 O
3,0,Omgaga Im sooo im gunna CRy Ive be...
4,0,i think mi bf is cheating on me TT
5,0,or i just worry too much
6,1,Juuuuuuuuuuuuuuuuussssst Chillin
7,0,Sunny Again Work Tomorrow ...
8,1,handed in my uniform today i miss you a...
9,1,hmmmm i wonder how she my number


## Text Vectorization

PARAMETERS IN TFIDVECTORIZER

decode_error : {‘strict’, ‘ignore’, ‘replace’} (default=’strict’)
Instruction on what to do if a byte sequence is given to analyze that contains characters not of the given encoding. By default, it is ‘strict’, meaning that a UnicodeDecodeError will be raised. Other values are ‘ignore’ and ‘replace’.

IF TOO MANY FEATURES, ADJUST HERE
max_df : float in range [0.0, 1.0] or int (default=1.0)
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

min_df : float in range [0.0, 1.0] or int (default=1)
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

DON'T CHANGE, BUT INCLUDE IN README THAT WE ARE DOING THE DEFAULTS AND WHY

norm : ‘l1’, ‘l2’ or None, optional (default=’l2’)
Each output row will have unit norm, either: * ‘l2’: Sum of squares of vector elements is 1. The cosine similarity between two vectors is their dot product when l2 norm has been applied. * ‘l1’: Sum of absolute values of vector elements is 1. See preprocessing.normalize

use_idf : boolean (default=True)
Enable inverse-document-frequency reweighting.

smooth_idf : boolean (default=True)
Smooth idf weights by adding one to document frequencies, as if an extra document was seen containing every term in the collection exactly once. Prevents zero divisions.


In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Consider changing parameters per above
vectorizer = TfidfVectorizer()

# Create feature matrix
x = vectorizer.fit_transform(twitter_df["Text"].tolist())
print(x.shape)

transformer = TfidfTransformer().fit(twitter_df["Text"].tolist())
tfidf_sample = tfidf_transformer.transform(bow_sample)
print(tfidf_sample)

(78612, 98482)


## Split into testing and training datasets

### Cross-validation?
What cross-validation does is splitting the training data into a certain number of training folds 
(with 75% of the training data) and a the same number of testing folds (with 25% of the training data), 
use the training folds to train the classifier, and test it against the testing folds to obtain performance 
metrics (see below). The process is repeated multiple times and an average for each of the metrics is calculated.

If your testing set is always the same, you might be overfitting to that testing set, which means you might 
be adjusting your analysis to a given set of data so much that you might fail to analyze a different set. 
Cross-validation helps prevent that.
The more data you have, the more folds you will be able to use.

kept test size at default, which is .25
just using part of the dataset now; use rest later

In [68]:
# Split into testing and training datasets
X_train, X_test, y_train, y_test = train_test_split(twitter_df["Text"], twitter_df["Sentiment"])


## Try Naive Bayes Classifier

We are going to use gridsearch to do cross-validation
Use complement MB b/c outperforms MNB (look at documentation)

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import ComplementNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
CONSIDER USING ADDITIONAL PARAMETERS FOR GRIDSEARCH AND ADDITIONAL FOLDS

In [66]:
# Create transformation pipeline
pipeline = Pipeline([
    ("vectorize", TfidfVectorizer()),
    ("classify", ComplementNB()),
])

# Set parameters for Gridsearch
parameters = {"vectorize__use_idf": (True, False),
              "vectorize__ngram_range": [(1,1), (1,2)],
              "vectorize__max_df" : (1, .8),
              "vectorize__norm": ("l1", "l2"),
              "classify__alpha": (.8, 1),
             }
grid = GridSearchCV(pipeline, cv = 5, param_grid = parameters, verbose = 1)

In [69]:
grid.fit(X_train,y_train)
# summarize results
print("\nBest Model: %f using %s" % (grid.best_score_, grid.best_params_))
print('\n')
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean: %f Stdev:(%f) with: %r" % (mean, stdev, param))

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed: 11.1min finished



Best Model: 0.770756 using {'classify__alpha': 0.8, 'vectorize__max_df': 0.8, 'vectorize__ngram_range': (1, 2), 'vectorize__norm': 'l2', 'vectorize__use_idf': True}


Mean: 0.480181 Stdev:(0.001880) with: {'classify__alpha': 0.8, 'vectorize__max_df': 1, 'vectorize__ngram_range': (1, 1), 'vectorize__norm': 'l1', 'vectorize__use_idf': True}
Mean: 0.480181 Stdev:(0.001880) with: {'classify__alpha': 0.8, 'vectorize__max_df': 1, 'vectorize__ngram_range': (1, 1), 'vectorize__norm': 'l1', 'vectorize__use_idf': False}
Mean: 0.480436 Stdev:(0.001584) with: {'classify__alpha': 0.8, 'vectorize__max_df': 1, 'vectorize__ngram_range': (1, 1), 'vectorize__norm': 'l2', 'vectorize__use_idf': True}
Mean: 0.480436 Stdev:(0.001584) with: {'classify__alpha': 0.8, 'vectorize__max_df': 1, 'vectorize__ngram_range': (1, 1), 'vectorize__norm': 'l2', 'vectorize__use_idf': False}
Mean: 0.532132 Stdev:(0.003343) with: {'classify__alpha': 0.8, 'vectorize__max_df': 1, 'vectorize__ngram_range': (1, 2), 'vectorize__n

## Apply algorithm (1 to begin)